In [2]:
import os
from pathlib import Path

# Preparation

First of all, make sure to download the corresponding medical ontologies to build the term dictionaries for bronco.
 - Treatments: [OPS](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html). Scroll to "OPS", 
 - Medications: [ATC](https://www.wido.de/publikationen-produkte/arzneimittel-klassifikation/)
 - Diagnosis: [ICD10GM](https://www.bfarm.de/DE/Kodiersysteme/Services/Downloads/_node.html)
 
In the config file for BRONCO `../conf/bronco.yaml`, modify the paths so they point the extracted ontologies.

We can already use xMEN to prepare the term dictionaries. In your terminal, navigate to the xMEN root folder and run:
 - `xmen dict conf/bronco.yaml --code dicts/atc2017_de.py --output temp/ --key atc`
 - `xmen dict conf/bronco.yaml --code dicts/ops2017.py --output temp/ --key ops`
 - `xmen dict conf/bronco.yaml --code dicts/icd10gm2017.py --output temp/ --key icd10gm`
 
Now use such dictionaries to build the indexes. For this example, we will use only SapBERT indexes and leave aside N-Gram:
 - `xmen index conf/bronco.yaml --dict temp/atc.jsonl --output temp/atc_index --sapbert`
 - `xmen index conf/bronco.yaml --dict temp/ops.jsonl --output temp/ops_index --sapbert`
 - `xmen index conf/bronco.yaml --dict temp/icd10gm.jsonl --output temp/icd10gm_index --sapbert`
 
Finally, load the adapted config file and the BRONCO150 dataset using BigBIO:

**openpyxl bioc**

In [3]:
from xmen.confhelper import load_config
config = load_config("../conf/bronco.yaml")

import datasets
path_to_data = r"C:\Users\admin\Desktop\BRONCO150" # paste here the path to the local data
ds = datasets.load_dataset(path = "bigbio/bronco", 
                           name = "bronco_bigbio_kb", 
                           data_dir=path_to_data)

ds

Using custom data configuration bronco_bigbio_kb-7a1d288ef98b5779
Found cached dataset bronco (C:/Users/admin/.cache/huggingface/datasets/bigbio___bronco/bronco_bigbio_kb-7a1d288ef98b5779/1.0.0/cab8fc4a62807688cb5b36df7a24eb7f364314862c4196f6ff2db3813f2fe68b)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 5
    })
})

# Candidate Generation
We will use `SapBERTLinker`, which uses a Transformer model to retrieve candidates with dense embeddings.

We could have also used `TFIDFNGramLinker` (see `xmen/notebooks/BioASQ_DisTEMIST.ipynb` for an example).

In [4]:
from notebook_util import analyze
from xmen.linkers import TFIDFNGramLinker, SapBERTLinker, EnsembleLinker
from xmen.linkers.util import filter_and_apply_threshold
from datasets import DatasetDict

In [22]:
embedding_model_name = 'cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR'
index_path = "../temp/atc_index/index/sapbert"

# Clear singleton to free up memory
SapBERTLinker.clear()
sapbert_linker = SapBERTLinker(
    embedding_model_name = embedding_model_name,
    index_base_path = index_base_path / 'sapbert',
    k = 1000
)

pred_sapbert = sapbert_linker.predict_batch(ds, batch_size=128)

[05/15/23 17:12:50] INFO     Loading hierarchical faiss index                                ]8;id=10869;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py\sap_bert_linker.py]8;;\:]8;id=782748;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/sap_bert_linker.py#148\148]8;;\

                    INFO     Loading index from                                                 ]8;id=16531;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=499567;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#64\64]8;;\
                             /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls                    
                             _en_es/index/sapbert/embed_faiss_hier.pickle                                          

[05/15/23 17:13:42] INFO     Loaded index of type <class 'faiss.swigfaiss.IndexHNSWFlat'> and   ]8;id=825163;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py\faiss_indexer.py]8;;\:]8;id=870349;file:///home/Florian.Borchert/miniconda3/envs/xmen_notebooks/lib/python3.10/site-packages/xmen/linkers/faiss_indexer.py#66\66]8;;\
                             size 1518833                                                                          

Map:   0%|          | 0/466 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [23]:
# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['validation'], pred_sapbert['validation'])

Perf@1 0.3634558093346574
Perf@2 0.5243296921549155
Perf@4 0.6593843098311817
Perf@8 0.7437934458788481
Perf@16 0.7864945382323734
Perf@32 0.8083416087388282
Perf@64 0.8262164846077458
